In [5]:
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv

In [6]:
# Retrieve variable values
aws_rds_host = os.getenv("DB_HOST")
aws_rds_port = os.getenv("DB_PORT")
aws_rds_name = os.getenv("DB_NAME")
aws_rds_user = os.getenv("DB_USER")
aws_rds_password = os.getenv("DB_PASSWORD")

In [12]:
def connect_to_db():
    """Connect to the database db_name and return the connection."""
    return psycopg2.connect(
            host=aws_rds_host,
            port=aws_rds_port,
            user=aws_rds_user,
            password=aws_rds_password,
            dbname=aws_rds_name
    )

In [ ]:
# Load variables from .env file
load_dotenv()

# Test Database connection
def test_connection():
    try:
        conn = connect_to_db()
        print("Connexion réussie !")
        conn.close()
    
    except Exception as e:
        print(f"Erreur de connexion : {e}")


test_connection()


Connexion réussie !


In [ ]:
# Creation of the table of cities with their GPS coordinates
def create_table_ville_gps():
    try:
        # Connection to the RDS base
        conn = connect_to_db()
        cur = conn.cursor()

        cur.execute("""
            CREATE TABLE IF NOT EXISTS villes (
                id SERIAL PRIMARY KEY,
                villes TEXT NOT NULL,
                longitude FLOAT NOT NULL,
                latitude FLOAT NOT NULL
            )
        """)

        conn.commit()
        cur.close()
        conn.close()
        print("Table créée avec succès !")

    except Exception as e:
        print(f"Erreur lors de la création de la table : {e}")

create_table_ville_gps()

Table créée avec succès !


In [ ]:
# Insertion of cities into the RDS database from the geolocalization.csv file
def geo_villes():

    df = pd.read_csv("geolocalisation.csv", sep=";")

    # Connection to the RDS base
    conn = connect_to_db()

    with conn.cursor() as cursor:
        for _, row in df.iterrows():
            cursor.execute("INSERT INTO villes (villes, longitude, latitude) VALUES (%s, %s, %s)", (row['adresse'], float(row['longitude']), float(row['latitude'])))
        
    conn.commit()
    conn.close()

geo_villes()

In [ ]:
# creation of the hotels table
def create_table_hotel():
    try:
        # Connection to the RDS base
        conn = connect_to_db()
        cur = conn.cursor()
        
        cur.execute("""
            CREATE TABLE IF NOT EXISTS hotels (
                id SERIAL PRIMARY KEY,
                villes TEXT NOT NULL,
                Nom_hotel TEXT NOT NULL,
                Note TEXT NOT NULL,
                URL TEXT NOT NULL,
                Description TEXT NOT NULL
            )
        """)

        conn.commit()
        cur.close()
        conn.close()
        print("Table créée avec succès !")

    except Exception as e:
        print(f"Erreur lors de la création de la table : {e}")

create_table_hotel()

Table créée avec succès !


In [ ]:
# inserting data into the hotels table from the hotels.csv file
def insert_hotel():
    df = pd.read_csv("hotels.csv", sep=";")

    # Connection to the RDS base
    conn = connect_to_db()
    
    with conn.cursor() as cursor:
        for _, row in df.iterrows():
            cursor.execute("INSERT INTO hotels (villes, Nom_hotel, Note, URL, Description) VALUES (%s, %s, %s, %s, %s)", (row['villes'], row['Nom hotel'], row['Note'], row['URL'], row['Description']))
        
    conn.commit()
    conn.close()

insert_hotel()